# Sprawozdanie z cw.2 Antoni Kois gr.2

In [ ]:
!pip3 install openpyxl seaborn scikit-learn statsmodels
import pandas as pd

# Wczytanie danych z pliku
df = pd.read_excel('../cw1/dane_przekrojowe_przykład.xlsx')

# Podstawowe statystyki opisowe
print(df.describe())

# Zadanie 1. Przygotowanie danych

### 1.1
Na podstawie współczynników zmienności i współczynników korelacji ocenić, czy któreś ze
zmiennych (wszystkich dostępnych) powinny zostać wyeliminowane przed przystąpieniem do
budowy modelu ekonometrycznego.

### 1.2
Zmodyfikować dostępny zbiór danych (rozważyć przekształcenia zmiennych, eliminację pewnych
obserwacji).

### 1.3
Dokonać podziału zbioru danych na zbiór uczący (90% obserwacji) i testowy (10% obserwacji).
Porównać statystyki na zbiorze uczącym i testowym. W jakim celu wykonywany jest ten podział? 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import variation
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Obliczenie współczynników zmienności
num_cols = ['year', 'price', 'mileage', 'tax', 'mpg', 'engineSize']
cv = df[num_cols].apply(lambda x: variation(x, nan_policy='omit'))
print("\nWspółczynniki zmienności dla naszego zbioru:")
print(cv)

# Obliczenie macierzy korelacji aby sprawdzić jak łątwiej sprawdzić jak one wyglądają
correlation_matrix = df[num_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Macierz korelacji")
plt.show()

In [ ]:
# Usuwanie obserwacji odstających na podstawie ceny (5 i 95 percentyl)
lower, upper = df['price'].quantile([0.05, 0.95])
df = df[(df['price'] >= lower) & (df['price'] <= upper)]

# Usunięcie obserwacji, gdzie engineSize = 0, ponieważ jest to prawdopodobnie zastąpienie wartości NaN
df = df[df['engineSize'] > 0]

# Dodanie nowej kolumny - wiek samochodu. Gdyż poprostu dla roku współczynnik zmienności jest mały i bez senus jest go porównywać
df['car_age'] = 2025 - df['year']
df.drop(columns=['year'], inplace=True)  # Usunięcie kolumny 'year', nie jest już nam potrzebna

# Usunięcie zmiennej 'mileage' (zbyt duża korelacja z rokiem, więc zostawiamy tylko jedną zmienną objaśniającą z tych dwóch)
df.drop(columns=['mileage'], inplace=True)

# Usunięcie wierszy, gdzie fuelType = Hybrid, gdyż wyraźnie widać, że tych aut ilościowo jest sporo mniej
df = df[df['fuelType'] != 'Hybrid']

# Usunięcie kolumny 'model', gdyż jest zmienną kategoryczną i ma zbyt wiele różnych wartości. Albo można "model" połączyć w mniejsze grupy i wtedy zmienić na faktory
df.drop(columns=['model'], inplace=True)

# Konwersja zmiennych kategorycznych na numeryczne (faktory)
df = pd.get_dummies(df, columns=['transmission', 'fuelType'], drop_first=True)

# Ponowne Sprawdzenie współczynników zmienności
num_cols = ['price', 'tax', 'mpg', 'engineSize', 'car_age']
cv = df[num_cols].apply(lambda x: variation(x, nan_policy='omit'))
print("\nWspółczynniki zmienności:")
print(cv)

# sprawdzenie czy nie ma kolumn kategorycznych
df.info()

In [ ]:
# Konwersja bool na int, ponieważ metoda OLS nie przyjmuje typu boolean
df[df.select_dtypes(['bool']).columns] = df.select_dtypes(['bool']).astype(int)

# Podział na zbiór uczący i testowy
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Porównanie statystyk zbiorów
print("\nStatystyki zbioru uczącego:")
print(train_df.describe())
print("\nStatystyki zbioru testowego:")
print(test_df.describe())


# Budowa modelu ekonometrycznego
X_train = train_df.drop(columns=['price'])
X_train[X_train.select_dtypes(['bool']).columns] = X_train.select_dtypes(['bool']).astype(int)  # Konwersja bool na int
y_train = train_df['price']
X_train = sm.add_constant(X_train)  # Dodanie stałej do modelu, w R funckja lm jest dodawan pod spodem w trakcie
model = sm.OLS(y_train, X_train).fit()

# Podsumowanie modelu
print(model.summary())

# Przetestowanie na danych testowych
X_test = test_df.drop(columns=['price'])
y_test = test_df['price']
X_test = sm.add_constant(X_test)  # Dodanie stałej
y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Średni błąd bezwzględny - Mean Absolute Error (MAE): {mae}")
print(f"Średni błąd kwadratowy - Mean Squared Error (MSE): {mse}")
print(f"R²: {r2}")


Podział na model uczący i uczony jest po to, by móc sprawdzić czy nasz model faktycznie działa odpowiednio, a do tego potrzebujemy danych, które model jeszcze "nie widział", gdyż istnieje ryzyko, że model działa znakomicie, ale tylko na danych które zna.

Po przeprowadzonym teście wynika, że średnio model myli się o 2458.84 £ w cenie.
MSE jest bardziej wrażliwy na duże błędy (przez podnoszenie do kwadratu), więc wysokie wartości mogą sugerować, że niektóre przewidywania są znacząco nietrafione.
R² (współczynnik determinacji) wynosi 0.827, co oznacza, że model wyjaśnia 82,7% wariancji cen samochodów, jest to dość spora ilość i wskazuje początkowo iż nasz model jest dobry

# 2 Budowa i weryfikacja modelu ekonometrycznego (na zbiorze uczącym)


### 2.1
Za pomocą MNK oszacować parametry modelu, w którym zmienną objaśnianą (Y) będzie cena, a zmiennymi objaśniającymi (Xi) wszystkie zmienne, które zostały wybrane w zadaniu 1.

### 2.2
Zapisać reszty modelu i wykonać kilka różnych testów normalności rozkładu reszt. Podać nazwy wykorzystywanych testów, ich hipotezy oraz interpretację wyników. Do czego potrzebne jest założenie o rozkładzie normalnym reszt?

In [ ]:
residuals = model.resid

# Test Shapiro-Wilka
# H0: Reszty mają rozkład normalny.
# H1: Reszty nie mają rozkładu normalnego.
from scipy.stats import shapiro
stat, p_value = shapiro(residuals)
print(f"Shapiro-Wilk test: stat={stat}, p={p_value}")


# Test Kołmogorowa-Smirnowa
# H0: Reszty mają rozkład normalny.
# H1: Reszty nie mają rozkładu normalnego.
from scipy.stats import kstest
stat, p_value = kstest(residuals, 'norm', args=(residuals.mean(), residuals.std()))
print(f"Kołmogorow-Smirnow test: stat={stat}, p={p_value}")

# Test Jarque-Bera
# H0: Reszty mają rozkład normalny.
# H1: Reszty nie mają rozkładu normalnego.
from statsmodels.stats.stattools import jarque_bera
stat, p_value, _, _ = jarque_bera(residuals)
print(f"Jarque-Bera test: stat={stat}, p={p_value}")


Jeśli wartość p w testach jest mniejsza niż 0.05, odrzucamy hipotezę o normalności reszt, co oznacza, że model może nie spełniać jednego z założeń MNK.
Normalność reszt jest istotnym założeniem modeli ekonometrycznych, ponieważ wpływa na jakości predykcji modelu.

Widzę, po wynikach, iż odrzucają one H0 mówiące o normalności. Jednak podejrzewam, że jest to błędna decyzja spowodowana prawdopodobnie zbyt dużą próbką danych. Dlatego ponawiam testy na losowo wybranych próbkach

In [ ]:
# Histogram reszt, by sprawdzić wizualnie czy reszty wyglądają na rozkład normalny
sns.histplot(residuals, kde=True, bins=30, stat="density", color="blue")

import statsmodels.api as sm

sm.qqplot(residuals, line='s')
plt.title("Wykres Q-Q dla reszt modelu")
plt.show()

sample_residuals = np.random.choice(residuals, 500, replace=False)  # Próbka 500 reszt

shapiro_test = shapiro(sample_residuals)
print(f"Shapiro-Wilk (próbka): stat={shapiro_test.statistic}, p={shapiro_test.pvalue}")

stat, p_value = kstest(sample_residuals, 'norm', args=(sample_residuals.mean(), sample_residuals.std()))
print(f"Kołmogorow-Smirnow test: stat={stat}, p={p_value}")

stat, p_value, _, _ = jarque_bera(sample_residuals)
print(f"Jarque-Bera test: stat={stat}, p={p_value}")


Teraz na podstawie testów można potwierdzić, że reszty mają rozkład normalny

### 2.3
Wybrać ostateczny podzbiór zmiennych objaśniających (na 2 sposoby):

- Wykorzystać w tym celu metodę krokową wsteczną (bazującą na teście istotności t-studenta). Przedstawić krótko ideę tej metody.

W tej metodzie zaczynamy od pełnego modelu (wszystkie zmienne) i usuwamy najmniej istotne (te z największym p-value w teście t-Studenta), aż zostaną tylko istotne zmienne.
Jeśli p-value zmiennej w teście t-Studenta jest większe niż 0.05, oznacza to, że zmienna nie jest istotna i powinna zostać usunięta.


- Wykorzystać kryteria informacyjne AIC, BIC.

Metoda opiera się na minimalizacji AIC i BIC. W skrócie: im niższe AIC/BIC, tym lepszy model.
Wybieramy kolumnę do usunięcia na podstawie p-value.
Jeśli AIC/BIC się poprawiły (zmniejszyły), zostawiamy zmienną usuniętą i kontynuujemy proces.
Jeśli AIC/BIC się pogorszyły, cofamy usunięcie i testujemy inną zmienną.

In [ ]:
# Sprawdzamy p-value dla każdej zmiennej w naszym dotychczaswoym modelu
print(model.pvalues)

# Widzę, że największe p-value ma transmission_Semi-Auto więc je usuwam i buduję model ponownie

In [ ]:

X_train_2_3_a = X_train.drop(columns=['transmission_Semi-Auto'])
model = sm.OLS(y_train, X_train_2_3_a).fit()

# Sprawdzamy nowe p-value i ponownie kasuję tą kolumnę z najwięksyzm p-value
print(model.pvalues)

In [ ]:
X_train_2_3_a = X_train_2_3_a.drop(columns=['fuelType_Petrol'])
model = sm.OLS(y_train, X_train_2_3_a).fit()
print(model.pvalues)

In [ ]:
X_train_2_3_a = X_train_2_3_a.drop(columns=['transmission_Manual'])
model = sm.OLS(y_train, X_train_2_3_a).fit()
print(model.pvalues)

Na tym etapie sądzę, że ten podzbiór zmiennychh opisujących będzie wystarczający:
- tax
- mpg
- engineSize
- car_age

Teraz przechodzę do drugiej metody AIC, BIC

In [33]:
# Przywracamy pierwotny model
model = sm.OLS(y_train, X_train).fit()

print(model.pvalues)
# Sprawdzamy początkowe wartości AIC i BIC
print(f"AIC: {model.aic}, BIC: {model.bic}")

const                     0.000000e+00
tax                       1.094083e-37
mpg                       0.000000e+00
engineSize                0.000000e+00
car_age                   0.000000e+00
transmission_Manual       2.870374e-73
transmission_Semi-Auto    5.895875e-01
fuelType_Petrol           2.493727e-26
dtype: float64
AIC: 163282.64955068944, BIC: 163339.10892881386


In [34]:
X_train_2_3_b = X_train.drop(columns=['transmission_Semi-Auto'])
model = sm.OLS(y_train, X_train_2_3_b).fit()

# Sprawdzamy nowe p-value i ponownie kasuję tą kolumnę z najwięksyzm p-value
print(model.pvalues)
print(f"AIC: {model.aic}, BIC: {model.bic}")

# AIC i BIC zmieniły się lekko w odpowiednią stronę

const                   0.000000e+00
tax                     1.053649e-37
mpg                     0.000000e+00
engineSize              0.000000e+00
car_age                 0.000000e+00
transmission_Manual    2.967069e-102
fuelType_Petrol         9.033157e-27
dtype: float64
AIC: 163280.9408280782, BIC: 163330.34278393706


In [35]:
X_train_2_3_b = X_train_2_3_b.drop(columns=['fuelType_Petrol'])
model = sm.OLS(y_train, X_train_2_3_b).fit()

print(model.pvalues)
print(f"AIC: {model.aic}, BIC: {model.bic}")

const                   0.000000e+00
tax                     1.838814e-30
mpg                     0.000000e+00
engineSize              0.000000e+00
car_age                 0.000000e+00
transmission_Manual    5.611715e-104
dtype: float64
AIC: 163393.76775864937, BIC: 163436.11229224267


In [36]:
# Wyniki zmieniły się na niekorzyśc więc przywracamy kolumnę i usuwamy inną
X_train_2_3_b['fuelType_Petrol'] = X_train['fuelType_Petrol']

X_train_2_3_b = X_train_2_3_b.drop(columns=['transmission_Manual'])
model = sm.OLS(y_train, X_train_2_3_b).fit()

print(model.pvalues)
print(f"AIC: {model.aic}, BIC: {model.bic}")

const              0.000000e+00
tax                2.892782e-31
mpg                0.000000e+00
engineSize         0.000000e+00
car_age            0.000000e+00
fuelType_Petrol    1.701652e-28
dtype: float64
AIC: 163740.30689844518, BIC: 163782.6514320385


In [37]:
# Wyniki zbów zmieniły się na niekorzyśc więc przywracamy kolumnę i usuwamy inną
X_train_2_3_b['transmission_Manual'] = X_train['transmission_Manual']

X_train_2_3_b = X_train_2_3_b.drop(columns=['tax'])
model = sm.OLS(y_train, X_train_2_3_b).fit()

print(model.pvalues)
print(f"AIC: {model.aic}, BIC: {model.bic}")

const                  0.000000e+00
mpg                    0.000000e+00
engineSize             0.000000e+00
car_age                0.000000e+00
fuelType_Petrol        1.672533e-19
transmission_Manual    7.507210e-96
dtype: float64
AIC: 163443.80388211738, BIC: 163486.1484157107


Wyniki na korzyść, usuwaliśmy już kolumny z największymi p-value, reszta ma sporo mniejsza wartości, więc to będzie nasz wynikowy zbiór zmiennych opisujących
- mpg
- engineSize
- car_age
- fuelType_Petrol
- transmission_Manual